In [1]:
# Importing Dependencies
import numpy as np
import gym
from gym.envs.box2d.car_racing import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from statistics import mean
from pyglet.window import key
import time
from tqdm import tqdm 
import os
from keras.utils import normalize

C:\Users\Piyush\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Function for generating data by playing
def start_playing(n_episodes):
    a = np.array( [0.0, 0.0, 0.0] )
    def key_press(k, mod):
        global restart
        if k==0xff0d: restart = True
        if k==key.LEFT:  a[0] = -1.0
        if k==key.RIGHT: a[0] = +1.0
        if k==key.UP:    a[1] = +1.0
        if k==key.DOWN:  a[2] = +0.8   
    def key_release(k, mod):
        if k==key.LEFT  and a[0]==-1.0: a[0] = 0
        if k==key.RIGHT and a[0]==+1.0: a[0] = 0
        if k==key.UP:    a[1] = 0
        if k==key.DOWN:  a[2] = 0
    
    env = CarRacing()
    env.render()
    env.viewer.window.on_key_press = key_press
    env.viewer.window.on_key_release = key_release
    
    for episode in range(n_episodes):
        prev_observation = env.reset()
        if episode==0:
            X = np.array(prev_observation.tolist())
            y = np.array([0.,0.,0.])
        restart = False
        while True:
            action = a
            observation, reward, done, info = env.step(action)
            X = np.vstack((X, prev_observation))
            y = np.vstack((y, action))
            prev_observation=observation
            env.render()
            if done or restart: break
        time.sleep(2)    
    np.save('train_X.npy', X)
    np.save('train_y.npy', y)    
    env.close()
    return X, y

In [3]:
# Playing the to create data
X, y=start_playing(20)

Track generation: 1121..1405 -> 284-tiles track
Track generation: 925..1162 -> 237-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1228..1538 -> 310-tiles track
Track generation: 1132..1419 -> 287-tiles track
Track generation: 1282..1607 -> 325-tiles track
Track generation: 1171..1472 -> 301-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1120..1404 -> 284-tiles track
Track generation: 1030..1291 -> 261-tiles track
Track generation: 1119..1403 -> 284-tiles track
Track generation: 1126..1412 -> 286-tiles track
Track generation: 1413..1770 -> 357-tiles track
Track generation: 1230..1541 -> 311-tiles track
Track generation: 1246..1562 -> 316-tiles track
Track generation: 1128..1414 -> 286-tiles track
Track generation: 1085..1365 -> 280-tiles track
Track generation: 1387..1738 -> 351-tiles track
Track generation: 992..1251 -> 259-tiles track
Track generation: 1029..1296 -> 267-tiles 

In [8]:
# Combining gameplay files
temp = np.load('train_data-549.4999999999591.npy')
tempX = [i[0] for i in temp]
tempy = [i[1] for i in temp]
X = np.array(tempX)
y = np.array(tempy)
for file in tqdm(os.listdir('Data/')):
    temp = np.load('Data/' + file)
    tempX = [i[0] for i in temp]
    tempy = [i[1] for i in temp]
    tempX = np.array(tempX)
    tempy = np.array(tempy)
    X  = np.concatenate([X, tempX], axis=0)
    y  = np.concatenate([y, tempy], axis=0)

np.save('X.npy', X)    

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.60it/s]


In [5]:
# Converting output to proper OneHot array
Y=[]
for i in range(len(y)):  
    if np.array_equal(y[i], np.array([0,1,0])):
        Y.append(np.array([0,0,1,0,0]))
    elif np.array_equal(y[i], np.array([0,0,0.8])):
        Y.append(np.array([0,0,0,1,0]))
    elif np.array_equal(y[i], np.array([1,0,0])):
        Y.append(np.array([0,1,0,0,0]))
    elif np.array_equal(y[i], np.array([-1,0,0])):
        Y.append(np.array([1,0,0,0,0]))
    else:
        Y.append(np.array([0,0,0,0,1]))      
      
np.save('y_modified.npy', Y)    

In [5]:
# Loading the data
X = np.load('train_X.npy')
y = np.load('y_modified.npy')
X = X.reshape(-1,96,96,3)
print(X.shape)
print(y.shape)

(20450, 96, 96, 3)
(20450, 5)


In [26]:
# Variation in generated data
left=np.array([1,0,0,0,0])
right=np.array([0,1,0,0,0])
straight=np.array([0,0,1,0,0])
stop=np.array([0,0,0,1,0])
no_key=np.array([0,0,0,0,1])
l=0
r=0
s=0
b=0
nk=0

for key in y:
    if np.array_equal(key, left):
        l+=1
    elif np.array_equal(key, right):
        r+=1
    elif np.array_equal(key, straight):
        s+=1
    elif np.array_equal(key, stop):
        b+=1
    elif np.array_equal(key, no_key):
        nk+=1    
      
print(l,r,s,b,nk)    

2694 1318 4443 179 11816


In [19]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (96,96,3)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(5, activation = "softmax"))

In [20]:
optimizer = Adam(lr=0.0001)
model.compile(optimizer = optimizer, loss = "categorical_crossentropy")

In [21]:
model.fit(X, y, epochs=10, validation_split=0.2, batch_size=600, verbose=1, shuffle=True)

Train on 16360 samples, validate on 4090 samples
Epoch 1/10
16360/16360 [==============================] - ETA: 45s - loss: 2.78 - ETA: 25s - loss: 2.73 - ETA: 19s - loss: 2.68 - ETA: 15s - loss: 2.66 - ETA: 13s - loss: 2.62 - ETA: 11s - loss: 2.59 - ETA: 10s - loss: 2.54 - ETA: 9s - loss: 2.5119 - ETA: 8s - loss: 2.471 - ETA: 8s - loss: 2.457 - ETA: 7s - loss: 2.435 - ETA: 6s - loss: 2.411 - ETA: 6s - loss: 2.390 - ETA: 5s - loss: 2.372 - ETA: 5s - loss: 2.355 - ETA: 4s - loss: 2.352 - ETA: 4s - loss: 2.352 - ETA: 3s - loss: 2.337 - ETA: 3s - loss: 2.319 - ETA: 2s - loss: 2.314 - ETA: 2s - loss: 2.296 - ETA: 2s - loss: 2.287 - ETA: 1s - loss: 2.276 - ETA: 1s - loss: 2.268 - ETA: 0s - loss: 2.267 - ETA: 0s - loss: 2.256 - ETA: 0s - loss: 2.254 - 12s 708us/step - loss: 2.2528 - val_loss: 1.2970
Epoch 2/10
16360/16360 [==============================] - ETA: 8s - loss: 2.045 - ETA: 8s - loss: 2.007 - ETA: 8s - loss: 2.014 - ETA: 7s - loss: 2.012 - ETA: 7s - loss: 2.015 - ETA: 7s - loss: 2

In [27]:
# Saving the Model
model.save('Models/first_modelv0')

In [ ]:
# Loading the Model
model.load('Models/first-model')

In [25]:
#scores = []
#choices = []
env = CarRacing()
for each_game in range(3):
    #score = 0
    #game_memory = []
    prev_obs = env.reset()
    for i in range(1000):
        env.render()
        out = model.predict(prev_obs.reshape(-1,96,96,3))[0]
        print(out)
        
        if out[0]<=0.45:
            out[0]=0  
        if out[1]<=0.2:
            out[1]=0
        if out[2]<=0.01:
            out[2]=0   
        if out[3]<=0.47:
            out[3]=0
        if out[4]<=9:
            out[4]=0    
        
        output = np.argmax(out)
        action = np.zeros(5)
        np.put(action, output, 1)
        #choices.append(action)
        if np.array_equal(action, np.array([0,0,1,0,0])):
            action=np.array([0,1,0])
        elif np.array_equal(action, np.array([0,0,0,1,0])):
            action=np.array([0,0,0.8])
        elif np.array_equal(action, np.array([0,1,0,0,0])):
            action=np.array([1,0,0])
        elif np.array_equal(action, np.array([1,0,0,0,0])):
            action=np.array([-1,0,0])
        else:
            action=np.array([0,0,0,0,0])
        
        new_observation, reward, done, info = env.step(action)
        print(action,"by nn")
        prev_obs = new_observation
        #game_memory.append([new_observation, action])
        #score+=reward
        if done: 
            break
    #scores.append(score)
    #print('Episode {} Score: {}'.format(each_game+1,score))
    env.close()
#print('Average Score:',sum(scores)/len(scores))

Track generation: 1101..1380 -> 279-tiles track
[4.7221249e-05 1.9628758e-04 1.5701357e-10 3.5686833e-03 9.9618787e-01]
[-1  0  0] by nn
[3.1089790e-19 1.2077008e-12 8.5848971e-24 1.5339471e-09 1.0000000e+00]
[-1  0  0] by nn
[1.0862435e-21 5.5307684e-14 9.2980333e-25 8.6731575e-07 9.9999917e-01]
[-1  0  0] by nn
[1.8810872e-10 3.9375454e-08 1.2517567e-12 1.2999316e-08 1.0000000e+00]
[-1  0  0] by nn
[7.3507184e-01 9.1152065e-02 1.5107028e-05 1.6647963e-01 7.2813691e-03]
[-1  0  0] by nn
[8.3602285e-01 4.5435719e-02 2.1362997e-05 1.1798692e-01 5.3307001e-04]
[-1  0  0] by nn
[9.6824932e-01 8.7599205e-03 7.9003312e-06 2.2968993e-02 1.3818704e-05]
[-1  0  0] by nn
[9.9396849e-01 9.5307000e-04 1.9674803e-07 5.0782389e-03 2.3777117e-08]
[-1  0  0] by nn
[9.9603146e-01 5.4239994e-04 2.3023529e-07 3.4259257e-03 4.0482437e-08]
[-1  0  0] by nn
[9.9490285e-01 1.0684681e-03 1.1498599e-06 4.0268153e-03 7.3183082e-07]
[-1  0  0] by nn
[9.8896605e-01 3.2656207e-03 7.7227232e-06 7.7370075e-03 2.354

[0.814459   0.10722415 0.01195067 0.06448615 0.00187995]
[-1  0  0] by nn
[0.8286083  0.10116516 0.01286161 0.05223804 0.00512694]
[-1  0  0] by nn
[0.81920767 0.10290135 0.01404934 0.05220523 0.01163647]
[-1  0  0] by nn
[0.815993   0.08034269 0.01995308 0.05019597 0.03351539]
[-1  0  0] by nn
[0.81071776 0.05810343 0.02001009 0.04686131 0.06430732]
[-1  0  0] by nn
[0.8236493  0.04067479 0.014635   0.03879822 0.08224273]
[-1  0  0] by nn
[0.83092636 0.03310649 0.01262354 0.03341248 0.08993114]
[-1  0  0] by nn
[0.83204997 0.03205341 0.0111258  0.03155968 0.09321108]
[-1  0  0] by nn
[0.83307004 0.02868696 0.01014186 0.03233327 0.09576783]
[-1  0  0] by nn
[0.8144788  0.03246695 0.01103737 0.03224618 0.1097706 ]
[-1  0  0] by nn
[0.8089731  0.03061273 0.01299056 0.04076448 0.10665926]
[-1  0  0] by nn
[0.8050322  0.03070541 0.01238961 0.0415264  0.11034638]
[-1  0  0] by nn
[0.80003023 0.03320189 0.01654265 0.04329813 0.10692716]
[-1  0  0] by nn
[0.7660308  0.04789697 0.02383314 0.04

[0.12858744 0.56346345 0.01818814 0.03719619 0.25256473]
[1 0 0] by nn
[0.14237753 0.50044054 0.02787422 0.03825463 0.29105318]
[1 0 0] by nn
[0.16196828 0.41810802 0.05164701 0.04620298 0.32207373]
[1 0 0] by nn
[0.19977918 0.31756777 0.08469459 0.05216914 0.3457893 ]
[1 0 0] by nn
[0.2559337  0.20350185 0.12132747 0.06523915 0.35399774]
[1 0 0] by nn
[0.36112425 0.13621014 0.11211437 0.06869925 0.32185203]
[0 1 0] by nn
[0.5052536  0.09363912 0.06631509 0.06137509 0.2734171 ]
[-1  0  0] by nn
[0.6567606  0.0577898  0.02571411 0.03805623 0.22167926]
[-1  0  0] by nn
[0.7486293  0.03918315 0.01177627 0.02730441 0.17310685]
[-1  0  0] by nn
[0.82010806 0.03052316 0.00551907 0.02062709 0.12322268]
[-1  0  0] by nn
[0.85711974 0.02714646 0.00352758 0.01716839 0.09503778]
[-1  0  0] by nn
[0.86828095 0.02904837 0.00346455 0.01783893 0.08136715]
[-1  0  0] by nn
[0.86378765 0.03123447 0.00466946 0.0235577  0.07675073]
[-1  0  0] by nn
[0.88758564 0.02688634 0.00316127 0.02116757 0.06119915]

[0.19857012 0.5502702  0.00358895 0.03328585 0.21428496]
[1 0 0] by nn
[0.18472244 0.55365664 0.00403564 0.03600624 0.22157899]
[1 0 0] by nn
[0.24425125 0.46531233 0.00459558 0.03418555 0.25165522]
[1 0 0] by nn
[0.22631833 0.43932864 0.00624693 0.03367531 0.29443088]
[1 0 0] by nn
[0.2211756  0.4516808  0.00927234 0.03476507 0.2831062 ]
[1 0 0] by nn
[0.2543062  0.34208632 0.01270108 0.04100923 0.34989712]
[1 0 0] by nn
[0.28482157 0.24093547 0.02186981 0.05385841 0.39851475]
[1 0 0] by nn
[0.3628696  0.15328309 0.02734457 0.06232516 0.39417762]
[0 1 0] by nn
[0.43569717 0.10396111 0.02388483 0.06826797 0.3681889 ]
[0 1 0] by nn
[0.53446716 0.08365627 0.01970308 0.07041021 0.29176325]
[-1  0  0] by nn
[0.5516571  0.05705618 0.02001642 0.0714623  0.29980794]
[-1  0  0] by nn
[0.66359997 0.05153583 0.0076029  0.04330922 0.23395206]
[-1  0  0] by nn
[0.67960244 0.06345727 0.00568239 0.04826938 0.20298846]
[-1  0  0] by nn
[0.7140882  0.06365517 0.00526756 0.04573454 0.17125456]
[-1  0  

[0.67740977 0.04425943 0.02058094 0.03598372 0.22176617]
[-1  0  0] by nn
[0.6506965  0.05302519 0.02124099 0.03234839 0.24268895]
[-1  0  0] by nn
[0.62071335 0.06062797 0.02992606 0.04152662 0.24720599]
[-1  0  0] by nn
[0.525434   0.11611973 0.04415837 0.05151073 0.2627772 ]
[-1  0  0] by nn
[0.44954708 0.18175285 0.05578717 0.05557609 0.25733683]
[0 1 0] by nn
[0.29146186 0.2833695  0.06131564 0.04589039 0.3179626 ]
[1 0 0] by nn
[0.20006792 0.3413027  0.056991   0.03938206 0.36225632]
[1 0 0] by nn
[0.2010184  0.3551092  0.04029961 0.033055   0.3705178 ]
[1 0 0] by nn
[0.18802348 0.39818618 0.03601634 0.03331189 0.34446213]
[1 0 0] by nn
[0.19167313 0.40745425 0.03401422 0.03720679 0.3296516 ]
[1 0 0] by nn
[0.17225835 0.3861597  0.0421075  0.04847208 0.35100237]
[1 0 0] by nn
[0.19831753 0.42228583 0.0277397  0.04232116 0.30933583]
[1 0 0] by nn
[0.22081572 0.3920229  0.02550198 0.04463452 0.31702492]
[1 0 0] by nn
[0.1992819  0.4227523  0.02302377 0.03614099 0.31880108]
[1 0 0] 

...

[0.6468962  0.21038488 0.00447356 0.01785194 0.12039343]
[-1  0  0] by nn
[0.64596224 0.21996978 0.00485199 0.01982013 0.10939578]
[-1  0  0] by nn
[0.64336    0.24465646 0.00631816 0.02378868 0.08187664]
[-1  0  0] by nn
[0.6571996  0.25125974 0.00715617 0.02644828 0.05793612]
[-1  0  0] by nn
[0.6187475  0.30667815 0.00720484 0.02621497 0.0411546 ]
[-1  0  0] by nn
[0.5990729  0.34768027 0.00657712 0.02548436 0.02118532]
[-1  0  0] by nn
[0.58942986 0.36360395 0.0062862  0.02560091 0.01507912]
[-1  0  0] by nn
[0.60103226 0.35958394 0.00498067 0.02363655 0.01076648]
[-1  0  0] by nn
[0.6106374  0.35832432 0.00320184 0.02030651 0.00752991]
[-1  0  0] by nn
[0.6107023  0.35393804 0.00306882 0.02632548 0.00596541]
[-1  0  0] by nn
[0.63491726 0.34250173 0.00124854 0.01853269 0.00279983]
[-1  0  0] by nn
[6.8826222e-01 2.9611707e-01 5.5341003e-04 1.3944475e-02 1.1227762e-03]
[-1  0  0] by nn
[7.1346056e-01 2.7631465e-01 2.3445686e-04 9.6225673e-03 3.6788249e-04]
[-1  0  0] by nn
[7.72376

[-1  0  0] by nn
[8.7193859e-01 1.1765991e-01 1.7710703e-03 7.9926485e-03 6.3784013e-04]
[-1  0  0] by nn
[8.6142355e-01 1.2815528e-01 1.9889430e-03 7.6495982e-03 7.8263099e-04]
[-1  0  0] by nn
[8.5657579e-01 1.3230886e-01 2.1773921e-03 8.0841128e-03 8.5375813e-04]
[-1  0  0] by nn
[0.8479096  0.1406387  0.00259434 0.00777646 0.00108096]
[-1  0  0] by nn
[0.8433307  0.1448947  0.00287792 0.00769102 0.0012057 ]
[-1  0  0] by nn
[0.83797306 0.14974843 0.00329895 0.00762682 0.00135273]
[-1  0  0] by nn
[0.83202785 0.15536004 0.00360603 0.00742987 0.00157627]
[-1  0  0] by nn
[0.8244009  0.16232158 0.00392111 0.00756764 0.0017887 ]
[-1  0  0] by nn
[0.8175094  0.16983877 0.00416795 0.00681426 0.00166963]
[-1  0  0] by nn
[0.8080578  0.17911592 0.00409247 0.0071127  0.00162114]
[-1  0  0] by nn
[0.80016965 0.18701223 0.004059   0.00720751 0.00155162]
[-1  0  0] by nn
[0.7951748  0.19270492 0.00379442 0.00701475 0.00131118]
[-1  0  0] by nn
[0.7925284  0.19592167 0.00363968 0.00678102 0.001

[9.8995399e-01 8.0049159e-03 1.1474990e-05 2.0294443e-03 5.9984579e-08]
[-1  0  0] by nn
[9.8954272e-01 8.1827072e-03 1.7719978e-05 2.2567122e-03 9.3121159e-08]
[-1  0  0] by nn
[9.8909724e-01 8.8737728e-03 2.2075592e-05 2.0068348e-03 8.6619835e-08]
[-1  0  0] by nn
[9.87367809e-01 1.07987663e-02 3.11184267e-05 1.80212338e-03
 1.11315664e-07]
[-1  0  0] by nn
[9.8701930e-01 1.1137985e-02 3.3638564e-05 1.8088883e-03 1.3449348e-07]
[-1  0  0] by nn
[9.8545724e-01 1.2415266e-02 4.7386391e-05 2.0799155e-03 2.8857065e-07]
[-1  0  0] by nn
[9.85670507e-01 1.16888955e-02 5.32817612e-05 2.58694403e-03
 3.40734715e-07]
[-1  0  0] by nn
[9.8423529e-01 1.2980759e-02 5.3442120e-05 2.7299246e-03 4.2473809e-07]
[-1  0  0] by nn
[9.8397022e-01 1.3445277e-02 6.2194835e-05 2.5215447e-03 6.5187527e-07]
[-1  0  0] by nn
[9.84649122e-01 1.34626785e-02 6.55927652e-05 1.82163203e-03
 9.78412913e-07]
[-1  0  0] by nn
[9.8294121e-01 1.5160406e-02 8.0312275e-05 1.8163397e-03 1.7107975e-06]
[-1  0  0] by nn
[9.

[-1  0  0] by nn
[8.85687888e-01 1.08729206e-01 5.28030505e-04 5.03367092e-03
 2.12212217e-05]
[-1  0  0] by nn
[9.3374497e-01 6.2286630e-02 1.3574473e-04 3.8294562e-03 3.1030875e-06]
[-1  0  0] by nn
[9.4888359e-01 4.7903232e-02 5.6193214e-05 3.1563442e-03 6.5882801e-07]
[-1  0  0] by nn
[9.6267724e-01 3.4956601e-02 2.3830522e-05 2.3421564e-03 1.3474725e-07]
[-1  0  0] by nn
[9.7352725e-01 2.4623379e-02 1.2032460e-05 1.8373367e-03 3.0343749e-08]
[-1  0  0] by nn
[9.7973633e-01 1.8609777e-02 6.6617922e-06 1.6472944e-03 9.3980503e-09]
[-1  0  0] by nn
[9.8490369e-01 1.3659220e-02 3.4027278e-06 1.4337186e-03 3.0329046e-09]
[-1  0  0] by nn
[9.8775464e-01 1.1109421e-02 2.0937759e-06 1.1338034e-03 1.2132998e-09]
[-1  0  0] by nn
[9.9047989e-01 8.5481117e-03 1.4051092e-06 9.7056525e-04 6.4848399e-10]
[-1  0  0] by nn
[9.9186397e-01 7.2418088e-03 9.7970053e-07 8.9322124e-04 3.3960693e-10]
[-1  0  0] by nn
[9.9297148e-01 6.0855155e-03 7.5026423e-07 9.4226404e-04 2.7250632e-10]
[-1  0  0] by n

[9.7098190e-01 2.5918802e-02 1.5607962e-04 2.9368056e-03 6.4308751e-06]
[-1  0  0] by nn
[9.6374255e-01 3.2502070e-02 2.2610452e-04 3.5195465e-03 9.5936593e-06]
[-1  0  0] by nn
[9.5989895e-01 3.5986591e-02 2.9924535e-04 3.8000559e-03 1.5130007e-05]
[-1  0  0] by nn
[9.5576119e-01 3.9255682e-02 3.8600148e-04 4.5688353e-03 2.8294036e-05]
[-1  0  0] by nn
[9.5979410e-01 3.5588037e-02 3.8637593e-04 4.2041130e-03 2.7366075e-05]
[-1  0  0] by nn
[9.5387644e-01 4.0580291e-02 4.5923900e-04 5.0345780e-03 4.9448667e-05]
[-1  0  0] by nn
[9.4271892e-01 5.0379843e-02 5.8315322e-04 6.2236264e-03 9.4366413e-05]
[-1  0  0] by nn
[9.2979914e-01 6.1852843e-02 8.0394611e-04 7.3877983e-03 1.5615385e-04]
[-1  0  0] by nn
[9.1045684e-01 8.0404200e-02 1.2053398e-03 7.6701897e-03 2.6345818e-04]
[-1  0  0] by nn
[8.87370408e-01 1.02689214e-01 1.59341283e-03 7.92256650e-03
 4.24413505e-04]
[-1  0  0] by nn
[8.87224913e-01 1.02698766e-01 1.53038907e-03 8.15106835e-03
 3.94866278e-04]
[-1  0  0] by nn
